In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
from scipy.signal import argrelextrema
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
def _is_monotonic(x):
    return np.all(np.diff(x) >= 0) or np.all(np.diff(x) <= 0)

def extract_imf_once(signal, max_sift_iter=100, sigma_threshold=0.1):
    """
    Try to extract one IMF by sifting.
    Returns imf, residue, converged (bool)
    """
    h = signal.copy()
    N = len(h)
    for _ in range(max_sift_iter):
        # find local maxima and minima indices
        max_idx = argrelextrema(h, np.greater)[0]
        min_idx = argrelextrema(h, np.less)[0]

        # If not enough extrema, break (can't form envelopes)
        if len(max_idx) + len(min_idx) < 3:
            return None, signal, False

        # include endpoints to make envelopes well-defined
        # prepare x for interpolation
        x = np.arange(N)

        # upper envelope
        xu = np.concatenate(([0], max_idx, [N-1]))
        yu = np.concatenate(([h[0]], h[max_idx], [h[-1]]))
        try:
            cs_up = CubicSpline(xu, yu)
            env_up = cs_up(x)
        except Exception:
            return None, signal, False

        # lower envelope
        xl = np.concatenate(([0], min_idx, [N-1]))
        yl = np.concatenate(([h[0]], h[min_idx], [h[-1]]))
        try:
            cs_low = CubicSpline(xl, yl)
            env_low = cs_low(x)
        except Exception:
            return None, signal, False

        mean_env = 0.5 * (env_up + env_low)

        prev_h = h
        h = h - mean_env

        # stopping criteria: small mean relative to signal amplitude and
        # number of zero crossings and extrema differ <=1 (empirical)
        zero_crossings = np.sum(np.abs(np.diff(np.sign(h))))  # counts sign changes (approx)
        num_extrema = len(argrelextrema(h, np.greater)[0]) + len(argrelextrema(h, np.less)[0])
        amp = np.max(np.abs(prev_h)) + 1e-16
        mean_norm = np.mean(np.abs(mean_env)) / amp

        if mean_norm < sigma_threshold and abs(zero_crossings - num_extrema) <= 2:
            return h, signal - h, True

    # if not converged, still return last h as IMF candidate (but mark not converged)
    return h, signal - h, False

def EMD(signal, max_imfs=10, max_sift_iter=100, sigma_threshold=0.1):
    """
    Simple EMD decomposition: iteratively extract IMFs until residue is monotonic or max_imfs reached.
    Returns: imfs (list of arrays), residue (array)
    """
    residue = signal.copy().astype(float)
    imfs = []
    for m in range(max_imfs):
        if _is_monotonic(residue):
            break
        imf, residue_new, converged = extract_imf_once(residue, max_sift_iter=max_sift_iter, sigma_threshold=sigma_threshold)
        if imf is None:
            # cannot extract further
            break
        imfs.append(imf)
        residue = residue_new
        # stop if residue becomes very small
        if np.allclose(residue, 0, atol=1e-12):
            break
    # append residue as last component if desired (optional)
    return np.array(imfs), residue

In [3]:
class RELM:
    def __init__(self, n_hidden=100, activation='tanh', C=1.0, random_state=None):
        self.n_hidden = int(n_hidden)
        self.activation = activation
        self.C = float(C)
        self.random_state = random_state
        self.is_fitted = False

    def _init_weights(self, n_features):
        rng = np.random.default_rng(self.random_state)
        self.W = rng.uniform(-1, 1, size=(self.n_hidden, n_features))
        self.b = rng.uniform(-1, 1, size=(self.n_hidden,))

    def _activation(self, X):
        if self.activation == 'sigmoid':
            X = np.clip(X, -500, 500)
            return 1.0 / (1.0 + np.exp(-X))
        if self.activation == 'tanh':
            return np.tanh(X)
        if self.activation == 'relu':
            return np.maximum(0.0, X)
        raise ValueError("Unknown activation")

    def fit(self, X, y):
        X = np.asarray(X); y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        N, d = X.shape
        self._init_weights(d)
        H = self._activation(X @ self.W.T + self.b)
        if N >= self.n_hidden:
            A = (np.eye(self.n_hidden) / self.C) + (H.T @ H)
            B = H.T @ y
            self.beta = np.linalg.solve(A, B)
        else:
            A = (np.eye(N) / self.C) + (H @ H.T)
            B = y
            self.beta = H.T @ np.linalg.solve(A, B)
        self.is_fitted = True
        return self

    def predict(self, X):
        H = self._activation(np.asarray(X) @ self.W.T + self.b)
        Y = H @ self.beta
        return Y.ravel() if Y.shape[1] == 1 else Y

# -------------------------
# GWO optimizer (classic)
# -------------------------
class GWO:
    def __init__(self, obj_func, lb, ub, dim, n_agents=12, n_iter=25, seed=42):
        self.obj = obj_func
        self.lb = np.array(lb, dtype=float)
        self.ub = np.array(ub, dtype=float)
        self.dim = dim
        self.n_agents = n_agents
        self.n_iter = n_iter
        self.rng = np.random.default_rng(seed)

    def optimize(self):
        wolves = self.rng.uniform(self.lb, self.ub, size=(self.n_agents, self.dim))
        fitness = np.array([self.obj(w) for w in wolves])
        idx = np.argsort(fitness)
        alpha, beta, delta = wolves[idx[0]].copy(), wolves[idx[1]].copy(), wolves[idx[2]].copy()
        f_alpha, f_beta, f_delta = float(fitness[idx[0]]), float(fitness[idx[1]]), float(fitness[idx[2]])
        for t in range(self.n_iter):
            a = 2 - 2 * (t / (self.n_iter - 1 + 1e-9))
            for i in range(self.n_agents):
                X = wolves[i].copy()
                for j in range(self.dim):
                    r1, r2 = self.rng.random(), self.rng.random()
                    A1 = 2 * a * r1 - a; C1 = 2 * r2
                    D_alpha = abs(C1 * alpha[j] - X[j]); X1 = alpha[j] - A1 * D_alpha

                    r1, r2 = self.rng.random(), self.rng.random()
                    A2 = 2 * a * r1 - a; C2 = 2 * r2
                    D_beta = abs(C2 * beta[j] - X[j]); X2 = beta[j] - A2 * D_beta

                    r1, r2 = self.rng.random(), self.rng.random()
                    A3 = 2 * a * r1 - a; C3 = 2 * r2
                    D_delta = abs(C3 * delta[j] - X[j]); X3 = delta[j] - A3 * D_delta

                    X[j] = (X1 + X2 + X3) / 3.0
                wolves[i] = np.clip(X, self.lb, self.ub)
            fitness = np.array([self.obj(w) for w in wolves])
            idx = np.argsort(fitness)
            if fitness[idx[0]] < f_alpha:
                alpha, f_alpha = wolves[idx[0]].copy(), float(fitness[idx[0]])
            if fitness[idx[1]] < f_beta:
                beta, f_beta = wolves[idx[1]].copy(), float(fitness[idx[1]])
            if fitness[idx[2]] < f_delta:
                delta, f_delta = wolves[idx[2]].copy(), float(fitness[idx[2]])
        return alpha, f_alpha


In [4]:
def create_multivariate_lagged_dataset(df, target_col, feature_cols, lag=3):
    data = df[feature_cols].values
    target_idx = feature_cols.index(target_col)
    X, y = [], []
    for i in range(lag, len(df)):
        X.append(data[i-lag:i].flatten())
        y.append(data[i, target_idx])
    return np.array(X), np.array(y)

def safe_mape(y_true, y_pred, min_denom=1.0):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    mask = np.abs(y_true) >= min_denom
    if np.sum(mask) == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100)

def sde(y_true, y_pred):
    return float(np.std(np.asarray(y_true) - np.asarray(y_pred)))

In [5]:
def decode_relm_params(position, Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    h_raw, logC, act_raw = position
    n_hidden = int(np.round(Hmin + np.clip(h_raw, 0, 1) * (Hmax - Hmin)))
    n_hidden = int(np.clip(n_hidden, Hmin, Hmax))
    C = 10.0 ** float(np.clip(logC, Cmin_log, Cmax_log))
    act_idx = int(np.round(np.clip(act_raw, 0, 2)))
    activation = ['tanh', 'sigmoid', 'relu'][act_idx]
    return n_hidden, C, activation

def make_relm_objective(X_train, y_train, X_val, y_val, random_state=42,
                        Hmin=20, Hmax=500, Cmin_log=-4, Cmax_log=4):
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)
    def objective(position):
        n_hidden, C, activation = decode_relm_params(position, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)
        try:
            model = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            model.fit(X_train_s, y_train)
            y_pred = model.predict(X_val_s)
            return float(np.sqrt(mean_squared_error(y_val, y_pred)))
        except Exception:
            return 1e6
    return objective

In [6]:
def emd_gwo_relm_pipeline(
    df,
    target_column,
    feature_columns,
    lag_steps=12,
    max_imfs=6,
    emd_max_sift_iter=100,
    emd_sigma_threshold=0.1,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=500,
    Cmin_log=-4, Cmax_log=4,
    max_step_eval=7
):
    """
    Returns dict with:
      - per_mode_info: list of parameter dictionaries per IMF
      - one_step_metrics: combined reconstructed one-step metrics
      - multistep_df: DataFrame with reconstructed multi-step metrics
      - imfs: array (n_imfs, L), residue: array (L)
      - y_true_test, y_pred_reconstructed
    """
    # 1) EMD decomposition of target
    signal = df[target_column].values.astype(float)
    imfs, residue = EMD(signal, max_imfs=max_imfs, max_sift_iter=emd_max_sift_iter, sigma_threshold=emd_sigma_threshold)
    n_imfs = 0 if imfs.size == 0 else imfs.shape[0]
    if n_imfs == 0:
        raise RuntimeError("EMD produced no IMFs. Try lowering sigma_threshold or increasing max_sift_iter.")

    L = signal.shape[0]
    # Our lagging removes first lag_steps samples; number of supervised samples = L - lag_steps
    n_samples = L - lag_steps
    if n_samples <= 0:
        raise ValueError("lag_steps too large relative to series length.")
    train_end = int(0.7 * n_samples)
    val_end = int(0.85 * n_samples)

    per_mode_info = []
    test_mode_preds = []
    test_mode_lengths = []

    # 2) For each IMF, run GWO to tune RELM, train final on train+val, predict test IMF
    for m in range(n_imfs):
        imf_series = imfs[m, :]
        df_mode = df.copy()
        df_mode[target_column] = imf_series

        X_mode, y_mode = create_multivariate_lagged_dataset(df_mode, target_column, feature_columns, lag=lag_steps)
        X_train, y_train = X_mode[:train_end], y_mode[:train_end]
        X_val, y_val     = X_mode[train_end:val_end], y_mode[train_end:val_end]
        X_test, y_test   = X_mode[val_end:], y_mode[val_end:]

        if len(X_train) < 5 or len(X_val) < 1 or len(X_test) < 1:
            print(f"[IMF {m+1}] insufficient data for train/val/test splits; skipping.")
            per_mode_info.append({"imf": m+1, "skipped": True})
            test_mode_preds.append(np.zeros_like(X_test[:,0]) if X_test.size else np.array([]))
            test_mode_lengths.append(len(X_test))
            continue

        # objective for GWO
        obj = make_relm_objective(X_train, y_train, X_val, y_val, random_state=random_state,
                                  Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)

        lb = np.array([0.0, Cmin_log, 0.0], dtype=float)  # [h_raw (0..1), logC, act_raw]
        ub = np.array([1.0, Cmax_log, 2.0], dtype=float)

        gwo = GWO(obj, lb, ub, dim=3, n_agents=gwo_agents, n_iter=gwo_iters, seed=random_state + m)
        best_pos, best_fit = gwo.optimize()

        n_hidden, C, activation = decode_relm_params(best_pos, Hmin=Hmin, Hmax=Hmax, Cmin_log=Cmin_log, Cmax_log=Cmax_log)

        # Train final on train+val
        scaler = StandardScaler()
        X_trval = np.vstack([X_train, X_val])
        y_trval = np.concatenate([y_train, y_val])
        X_trval_s = scaler.fit_transform(X_trval)
        X_test_s = scaler.transform(X_test)

        model_final = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
        model_final.fit(X_trval_s, y_trval)
        y_pred_test_imf = model_final.predict(X_test_s)

        per_mode_info.append({
            "imf": m+1,
            "best_val_rmse": float(best_fit),
            "n_hidden": int(n_hidden),
            "C": float(C),
            "activation": activation,
            "test_len": int(len(y_test))
        })

        test_mode_preds.append(y_pred_test_imf)
        test_mode_lengths.append(len(y_test))

    # 3) Align predicted IMF test sequences and reconstruct by summation
    n_test = None
    for ln in test_mode_lengths:
        if ln:
            n_test = ln
            break
    if n_test is None:
        raise RuntimeError("No valid test data across IMFs.")

    stacked = []
    for arr in test_mode_preds:
        a = np.asarray(arr)
        if len(a) == n_test:
            stacked.append(a)
        elif len(a) == 0:
            stacked.append(np.zeros(n_test))
        elif len(a) < n_test:
            stacked.append(np.concatenate([a, np.zeros(n_test - len(a))]))
        else:
            stacked.append(a[:n_test])
    stacked = np.array(stacked)  # (n_imfs, n_test)

    # reconstructed forecast is sum of predicted IMFs + predicted residue (we didn't model residue)
    y_pred_reconstructed = np.sum(stacked, axis=0)
    # true test target values: original df target aligned after lag + val_end
    y_true_test = df[target_column].values[lag_steps + val_end : lag_steps + val_end + n_test]

    # 4) One-step metrics
    one_mae = mean_absolute_error(y_true_test, y_pred_reconstructed)
    one_rmse = np.sqrt(mean_squared_error(y_true_test, y_pred_reconstructed))
    one_mape = safe_mape(y_true_test, y_pred_reconstructed)
    one_sdev = sde(y_true_test, y_pred_reconstructed)

    one_step_metrics = {"MAE": float(one_mae), "RMSE": float(one_rmse), "MAPE (%)": float(one_mape) if not np.isnan(one_mape) else np.nan, "SDE": float(one_sdev)}

    # 5) Multi-step direct predictions: retrain per-IMF on TRAIN only with tuned params and predict step-ahead, then sum
    multistep_rows = []
    for step in range(1, max_step_eval + 1):
        mode_step_preds = []
        valid = True
        for info in per_mode_info:
            if info.get("skipped", False):
                mode_step_preds.append(np.zeros(max(0, n_test - step)))
                continue
            midx = info["imf"] - 1
            imf_series = imfs[midx, :]
            df_mode = df.copy(); df_mode[target_column] = imf_series
            X_mode, y_mode = create_multivariate_lagged_dataset(df_mode, target_column, feature_columns, lag=lag_steps)
            X_train, y_train = X_mode[:train_end], y_mode[:train_end]
            X_test_all, y_test_all = X_mode[val_end:], y_mode[val_end:]
            if X_test_all.shape[0] <= step:
                valid = False
                break
            X_test_step = X_test_all[:-step]
            # fit on TRAIN only with tuned params
            n_hidden = info["n_hidden"]
            C = info["C"]
            activation = info["activation"]
            scaler_train = StandardScaler()
            X_train_s = scaler_train.fit_transform(X_train)
            X_test_step_s = scaler_train.transform(X_test_step)
            model_step = RELM(n_hidden=n_hidden, activation=activation, C=C, random_state=random_state)
            model_step.fit(X_train_s, y_train)
            y_pred_step_imf = model_step.predict(X_test_step_s)
            mode_step_preds.append(y_pred_step_imf)
        if not valid:
            break
        mode_step_preds = np.array(mode_step_preds)  # (n_imfs, n_test-step)
        y_pred_step_recon = np.sum(mode_step_preds, axis=0)
        y_true_step = df[target_column].values[lag_steps + val_end + step : lag_steps + val_end + step + y_pred_step_recon.shape[0]]
        L_true = len(y_true_step); L_pred = len(y_pred_step_recon)
        m = min(L_true, L_pred)
        if m == 0:
            break
        y_true_step = y_true_step[:m]; y_pred_step_recon = y_pred_step_recon[:m]
        multistep_rows.append({
            "Step": step,
            "MAE": float(mean_absolute_error(y_true_step, y_pred_step_recon)),
            "RMSE": float(np.sqrt(mean_squared_error(y_true_step, y_pred_step_recon))),
            "MAPE (%)": float(safe_mape(y_true_step, y_pred_step_recon)),
            "SDE": float(sde(y_true_step, y_pred_step_recon))
        })

    multistep_df = pd.DataFrame(multistep_rows)

    return {
        "per_mode_info": per_mode_info,
        "one_step_metrics": one_step_metrics,
        "multistep_df": multistep_df,
        "imfs": imfs,
        "residue": residue,
        "y_true_test": y_true_test,
        "y_pred_reconstructed": y_pred_reconstructed
    }

In [7]:
feature_columns = ['AirTemp','Azimuth','CloudOpacity','DewpointTemp','Dhi','Dni','Ebh',
                   'WindDirection10m','Ghi','RelativeHumidity','SurfacePressure','WindSpeed10m']
df = pd.read_csv('/Users/hrishityelchuri/Documents/windPred/raw/8.52 hrishit data.csv')

df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd'])
df['PeriodStart'] = pd.to_datetime(df['PeriodStart'])
df = df.sort_values('PeriodEnd')

In [8]:
res = emd_gwo_relm_pipeline(
    df,
    target_column='WindSpeed10m',
    feature_columns=feature_columns,
    lag_steps=12,
    max_imfs=6,
    emd_max_sift_iter=100,
    emd_sigma_threshold=0.1,
    gwo_agents=12,
    gwo_iters=25,
    random_state=42,
    Hmin=20, Hmax=400,
    Cmin_log=-4, Cmax_log=4,
    max_step_eval=7
)
print("Per-IMF best params:")
print(pd.DataFrame(res['per_mode_info']))
print("\nOne-step reconstructed metrics:")
print(res['one_step_metrics'])
print("\nMulti-step reconstructed metrics:")
print(res['multistep_df'].to_string(index=False))

Per-IMF best params:
   imf  best_val_rmse  n_hidden           C activation  test_len
0    1       0.148430       352    0.000598       relu     20612
1    2       0.126941       395  734.861982       relu     20612
2    3       0.108588       397    0.003212       relu     20612
3    4       0.056994       392    0.002411       relu     20612
4    5       0.045601       392    0.002007       relu     20612
5    6       0.047560       395    0.000100       relu     20612

One-step reconstructed metrics:
{'MAE': 2.8241125621226737, 'RMSE': 3.0718726274595465, 'MAPE (%)': 104.12088934286336, 'SDE': 1.2087342018424205}

Multi-step reconstructed metrics:
 Step      MAE     RMSE   MAPE (%)      SDE
    1 2.841049 3.111382 104.771240 1.270654
    2 2.844974 3.159764 104.244214 1.384897
    3 2.859433 3.213632 103.785665 1.503776
    4 2.877869 3.264978 103.457545 1.610574
    5 2.892675 3.309902 102.921557 1.699831
    6 2.901033 3.347242 102.238476 1.771522
    7 2.902984 3.376430 101.46402